# Keras with MirroredStrategy
In this lab, you will take the solution you created in Lab 03b and implement MirroredStrategy.  You will take a Keras Sequential Model and convert to estimator, then distribute with MirroredStrategy.

It will be easiest to take the results you had from 4.2b and replace the model with the Keras Sequential Model.

Helpful references:
- https://www.tensorflow.org/guide/keras#multiple_gpus
- https://keras.io/losses/#sparse_categorical_crossentropy

In [1]:
import os
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

/usr/lib/python3/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Data Pipeline
Reuse data pipeline components from previous labs.

In [2]:
def parse_tfrecord(example):
    feature={'idx': tf.FixedLenFeature((), tf.int64),
             'label': tf.FixedLenFeature((), tf.int64),
             'image': tf.FixedLenFeature((), tf.string, default_value="")}
    parsed = tf.parse_single_example(example, feature)
    image = tf.decode_raw(parsed['image'],tf.float64)
    image = tf.cast(image,tf.float32)
    image = tf.reshape(image,[32,32,3])
    return image, parsed['label']

def image_scaling(x):
    return tf.image.per_image_standardization(x)

def distort(x):
    x = tf.image.resize_image_with_crop_or_pad(x, 40, 40)
    x = tf.random_crop(x, [32, 32, 3])
    x = tf.image.random_flip_left_right(x)
    return x

In [3]:
def dataset_input_fn(params):
    dataset = tf.data.TFRecordDataset(
        params['filenames'],num_parallel_reads=params['threads'])
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=params['threads'])
    dataset = dataset.map(lambda x,y: (image_scaling(x),y),num_parallel_calls=params['threads'])
    if params['mode']==tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.map(lambda x,y: (distort(x),y),num_parallel_calls=params['threads'])
        dataset = dataset.shuffle(buffer_size=params['shuffle_buff'])
    dataset = dataset.repeat()
    dataset = dataset.batch(params['batch'])
    dataset = dataset.prefetch(8*params['batch'])
    return dataset

In [4]:
train_files = !ls ./data/cifar10_data_00*.tfrecords
eval_files   = !ls ./data/cifar10_data_01*.tfrecords

train_params = {'filenames': train_files,
                'mode': tf.estimator.ModeKeys.TRAIN,
                'threads': 16,
                'shuffle_buff': 10000,
                'batch': 200}

eval_params  = {'filenames': eval_files,
                'mode': tf.estimator.ModeKeys.EVAL,
                'threads': 8,
                'batch': 200}

train_spec = tf.estimator.TrainSpec(input_fn=lambda: dataset_input_fn(train_params),max_steps=20000)
eval_spec  = tf.estimator.EvalSpec(input_fn=lambda: dataset_input_fn(eval_params),steps=10,throttle_secs=60)

### DO: Setup Keras model

In [5]:
num_classes = 10
model_params  = {'drop_out': 0.2, 'dense_units': 1024, 'learning_rate': 1e-3}

In [6]:
model = Sequential()
model.add(Conv2D(128, (5, 5), padding='same', input_shape=[32, 32, 3]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model.add(Conv2D(128, (5,5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(model_params['dense_units']))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Dropout(model_params['drop_out']))
model.add(Activation('softmax'))

opt = tf.train.GradientDescentOptimizer(model_params['learning_rate'])

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)

In [7]:
distribution = tf.contrib.distribute.MirroredStrategy(num_gpus=8)
config = tf.estimator.RunConfig(save_checkpoints_secs = 300,
                                keep_checkpoint_max = 5,
                                session_config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True),
                                train_distribute = distribution)

In [8]:
name = 'cnn_model/keras_model_'
name = name + 'dense' + str(model_params['dense_units']) + '_'
name = name + 'drop' + str(model_params['drop_out']) + '_'
name = name + 'lr' + str(model_params['learning_rate']) + '_'
name = name + time.strftime("%Y%m%d%H%M%S")
model_dir  = os.path.join('./',name)

print(model_dir)

./cnn_model/keras_model_dense1024_drop0.2_lr0.001_20180906093912


### DO: Create estimator from Keras Model

In [15]:
def my_accuracy(labels, predictions):
    pred_values = tf.argmax(predictions[model.output_names[0]], axis=1)
    return {'accuracy': tf.metrics.accuracy(labels, pred_values)}

In [16]:
_estimator = keras.estimator.model_to_estimator(
  keras_model=model, config=config, model_dir=model_dir)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_model_dir': './cnn_model/keras_model_dense1024_drop0.2_lr0.001_20180906093912', '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f54384371d0>, '_keep_checkpoint_every_n_hours': 10000, '_device_fn': None, '_save_checkpoints_secs': 300, '_save_checkpoints_steps': None, '_master': '', '_global_id_in_cluster': 0, '_task_id': 0, '_save_summary_steps': 100, '_is_chief': True, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7f54384370f0>, '_task_type': 'worker', '_service': None, '_session_config': allow_soft_placement: true
log_device_placement: true
, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_log_step_count_steps': 100}


In [17]:
estimator = tf.contrib.estimator.add_metrics(_estimator, my_accuracy)

INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_model_dir': './cnn_model/keras_model_dense1024_drop0.2_lr0.001_20180906093912', '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f563e1acda0>, '_keep_checkpoint_every_n_hours': 10000, '_device_fn': None, '_save_checkpoints_secs': 300, '_save_checkpoints_steps': None, '_evaluation_master': '', '_global_id_in_cluster': 0, '_task_id': 0, '_save_summary_steps': 100, '_is_chief': True, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7f563e1acdd8>, '_service': None, '_task_type': 'worker', '_session_config': allow_soft_placement: true
log_device_placement: true
, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_log_step_count_steps': 100}


In [12]:
# Run on 8 GPUs
estimator.train(input_fn=lambda: dataset_input_fn(train_params),max_steps=1000)

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:batch_all_reduce invoked for batches size = 12 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_smal

In [18]:
# Run on 1 GPU
estimator.evaluate(input_fn=lambda: dataset_input_fn(eval_params),steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-06-09:44:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/keras_model_dense1024_drop0.2_lr0.001_20180906093912/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-09-06-09:44:30
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.231, global_step = 1000, loss = 2.2412996
INFO:tensorflow:Saving 'checkpoint_path' summar

{'accuracy': 0.231, 'global_step': 1000, 'loss': 2.2412996}